In [1]:
import pdfplumber
import pandas as pd
import numpy as np
import os
import textEx
from datetime import datetime

In [25]:
def readAllXinXingPDF(dir:str) -> pd.DataFrame:
    column_names = ['重量', '貨名', '單價', '倉位', '重量', '貨名', '單價', '倉位']
    all_pages_df = pd.DataFrame(columns=column_names)
    all_pages_df.insert(4, 'date',None) 
    all_pages_df.insert(9, 'date2',None) 
    for filename in os.listdir(dir):
        if filename.endswith('.pdf'):
            file_path = os.path.join(dir, filename)
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        try:
                            df = pd.DataFrame(table[1:],columns=column_names)
                            df = df.replace("",np.nan).fillna(value=np.nan)
                            df = df.dropna(axis=1, how='all')
                            df = df.dropna(axis=0)
                            df.insert(4, 'date', filename.replace("報價單","").replace(".pdf","")) 
                            df.insert(9, 'date2', filename.replace("報價單","").replace(".pdf","")) 
                        except:
                            print(f"Error 1 merging data from {filename},{page}")
                        try:
                            all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                        except:
                            print(f"Error merging data from {filename},{page}")
    df1 = all_pages_df.iloc[:, :5]
    df2 = all_pages_df.iloc[:, 5:10]
    df2.columns = df1.columns
    dfall = pd.concat([df1, df2], axis=0).reset_index(drop=True)
    df_cleaned = dfall.dropna(subset=["貨名"], how='any').reset_index(drop=True)
    

    return df_cleaned

In [26]:
df=readAllXinXingPDF(r"C:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\quotation\新興")

In [27]:
df

,重量,貨名,單價,倉位,date
0,抄碼,巴西 牛腩(PUL),19.5P,光1,02-01-2024
1,抄碼,巴西 牛冧(GJ),20.0P,光1,02-01-2024
2,抄碼,巴西 西冷(NATURAFRIG),21.0P,光1,02-01-2024
3,抄碼,巴西 西冷(MASTERBOI),23.0P,光1,02-01-2024
4,抄碼,巴西 西冷(MINERVA),23.8P,光1,02-01-2024
...,...,...,...,...,...
2141,10K,比利時 粗直薯條(TOMEX)(2.5Kx4包)(10x10mm),清,光1,30-10-2023
2142,10K,比利時 扒房薯條(TOMEX)(2.5Kx4包)(10x20mm),6.8P,光1,30-10-2023
2143,10K,比利時 辣薯角(TOMEX)(2.5Kx4包),7.5P,光1,30-10-2023
2144,抄碼,紐西蘭 羊肩架 法式4支 (BLUE SKY),77.0P,光1,30-10-2023


In [28]:
def _specToList(spec :list) -> list:
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""
    if spec is not None:
        spec = [item for item in spec if item is not None]
        spec.sort()
        if len(spec) > 0:
            spec1 = spec[0]
        if len(spec) > 1:
            spec2 = spec[1]
        if len(spec) > 2:
            spec3 = spec[2]
        if len(spec) > 3:
            spec4 = spec[3]
        if len(spec) > 4:
            spec5 = spec[4]
        if len(spec) > 5:
            spec6 = spec[5]
    else:
        spec1=spec2=spec3=spec4=spec5=spec6=""

    return [spec1,spec2,spec3,spec4,spec5,spec6]
        

def getXinXingQuote(df: pd.DataFrame) -> pd.DataFrame:
    df['specs'] = df['貨名'].apply(textEx.getSpec)  
    df['brand'] = df['貨名'].apply(textEx.getBrand)
    for index, row in df.iterrows():
        if row['brand'] is not None:
            df.at[index, 'productName'] = row['貨名'].replace(row['brand'], '')
        else:
            df.at[index, 'productName'] = row['貨名']
    df['productTag'] = df['貨名'].apply(textEx.getProduct)
    df['supplier'] = '新兴'
    df['category'] = df['貨名'].apply(textEx.getCategory)
    df['packing']=df['重量']
    df['origin'] = df['貨名'].apply(textEx.getCountry)
    df['effectiveDate'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)
    df[['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6']] = df['specs'].apply(lambda x: pd.Series(_specToList(x)))
    df['price'] = df['單價']
    df['weightUnit'] = df['單價'].astype(str).apply(textEx.getWeightUnit)
    df['倉位'] = df['倉位'].astype(str)
    df['warehouse']=df['倉位'].apply(textEx.getWarehoue)
    df['notes']= None

    df = df.explode('warehouse')
    df['warehouse'] = df['warehouse'].astype(str)

    required_columns = [
    'productName', 'productTag', 'supplier', 'category', 'packing',
    'origin', 'brand', 'effectiveDate', 'spec1', 'spec2', 'spec3',
    'spec4', 'spec5', 'spec6', 'price', 'weightUnit', 'warehouse', 'notes'
    ]
    df=df[required_columns]
    return df

In [29]:
dfout=getXinXingQuote(df)

c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

...

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found


c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
c:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\textEx.py:62: UserWarning: DataF

no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is

In [31]:
dfout.to_csv("新兴.csv")